In [2]:
!pip install --quiet -U langchain langchain-openai
%pip install --quiet --upgrade langchain langchain-community langchain-chroma

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드
load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")
langchain_project = os.getenv("LANGCHAIN_PROJECT")
langchain_tracing = os.getenv("LANGCHAIN_TRACING_V2")

print(f"OpenAI API: {openai_api_key[:4]}********")
print(f"LangChain API Key: {langchain_api_key[:4]}********")

os.environ["OPENAI_API_KEY"] = openai_api_key
os.environ["LANGCHAIN_PROJECT"] = langchain_project
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_TRACING_V2"] = langchain_tracing


OpenAI API: sk-p********
LangChain API Key: lsv2********


LangSmith Setup (LLM Logging)


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("Hello, world!")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 9, 'prompt_tokens': 11, 'total_tokens': 20, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e5e963eb-b157-462f-81b7-02882559c41b-0', usage_metadata={'input_tokens': 11, 'output_tokens': 9, 'total_tokens': 20})

Starts from now

In [5]:
import os
import json
import getpass

# Set your OpenAI API key
#os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


In [6]:
#cd /content/drive/MyDrive/24fall/tenten/tenten-ai

In [7]:
# 시간 형식을 변환하는 함수
def ms_to_minutes_seconds_str(milliseconds):
    seconds = milliseconds // 1000  # 총 초
    minutes = seconds // 60  # 분 계산
    remaining_seconds = seconds % 60  # 남은 초 계산
    return f"{minutes}분 {remaining_seconds}초"  # 출력 형식

# 파일 로드
with open('transcription.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

utterances = data['utterances']

# 발화를 Document 객체로 생성
documents = []
for utterance in utterances:
    content = utterance['msg']
    metadata = {
        'start_at': utterance['start_at'],  # 밀리초 단위
        'duration': utterance['duration'],   # 밀리초 단위
        'speaker': utterance['spk'],
        'speaker_type': utterance['spk_type']
    }
    documents.append(Document(page_content=content, metadata=metadata))

# 시작 시간 기준으로 정렬
documents = sorted(documents, key=lambda doc: doc.metadata['start_at'])

In [10]:
full_text = ''
for doc in documents:
    start_time_str = ms_to_minutes_seconds_str(doc.metadata['start_at'])
    full_text += f"[{start_time_str}] {doc.page_content}\n"

# 언어 모델 초기화
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

In [18]:
import re
summary_prompt = PromptTemplate(
    input_variables=["text"],
    template="""
아래는 상담 세션의 전체 스크립트입니다. 이 스크립트를 시간의 흐름에 따라 **주제별**로 요약해 주세요.

다음 조건을 따라 주세요:
1. 주제는 각 시간 구간에서 **한 가지 주요 질문** 또는 **하나의 조언**으로 요약해 주세요. 여러 주제가 섞이지 않도록 주의하세요.
2. 각 주제에 대해 시작 시간과 종료 시간을 포함해 주세요. 시간은 해당 주제가 처음 등장한 스크립트의 시작 시간과 마지막으로 등장한 스크립트의 시작 시간을 사용하여 계산됩니다.
3. 각 시간 구간에 해당하는 요약은 아래 중 **하나**입니다:
   - 질문인 경우: **Q.** 형식으로 질문을 요약해 주세요.
   - 행동을 변경해야 하는 경우: **[바꾸세요]** 형식으로 요약해 주세요.
   - 권장 사항인 경우: **[이렇게 하세요]** 형식으로 요약해 주세요.

예시:
- 0분 42초 ~ 3분 15초
  - Q. 당뇨 약을 일주일 간 끊어도 될까요?

- 5분 0초 ~ 7분 30초
  - [바꾸세요] 당뇨 약은 중단하지 마세요.

- 10분 15초 ~ 12분 45초
  - [이렇게 하세요] 약 복용을 지속하세요.

스크립트:
{text}

요약:
"""
)
# $0.17
# 요약 체인 생성
summary_chain = LLMChain(llm=llm, prompt=summary_prompt)

# 전체 스크립트에 대해 요약 실행
session_summary = summary_chain.run(text=full_text)
print(session_summary)

# LLM의 출력을 파싱하여 주제와 시간 정보 추출
subtopics = re.findall(r'- (\d+분 \d+초) ~ (\d+분 \d+초)\n\s*- (.+?)(?=\n- \d+분|\Z)', session_summary, re.DOTALL)

# 임베딩 및 벡터스토어 초기화
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents, embeddings)

KeyboardInterrupt: 

In [15]:
# 리트리버 설정
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})


In [16]:
# 각 주제에 대해 처리
for idx, (start_time_str, end_time_str, content) in enumerate(subtopics):
    print(f"\n### 주제 {idx + 1}: {start_time_str} ~ {end_time_str} ###\n")
    subtopic_lines = content.strip().split('\n')
    subtopic_lines = [line.strip() for line in subtopic_lines if line.strip() != '']
    for line in subtopic_lines:
        print(line)
    
    # 시간 문자열을 밀리초로 변환하는 함수
    def time_str_to_ms(time_str):
        match = re.match(r'(\d+)분 (\d+)초', time_str)
        if match:
            minutes = int(match.group(1))
            seconds = int(match.group(2))
            return (minutes * 60 + seconds) * 1000
        else:
            return 0

    start_time_ms = time_str_to_ms(start_time_str)
    end_time_ms = time_str_to_ms(end_time_str)
    
    # 해당 시간 범위 내의 문서 필터링
    relevant_docs = [
        doc for doc in documents
        if start_time_ms <= doc.metadata['start_at'] <= end_time_ms
    ]
    
    # 시작 시간 기준으로 정렬
    sorted_relevant_docs = sorted(relevant_docs, key=lambda doc: doc.metadata['start_at'])
    
    # 출력 형식 지정
    start_at_and_context = "\n\n".join(
        [f"{ms_to_minutes_seconds_str(doc.metadata['start_at'])}: {doc.page_content}" for doc in sorted_relevant_docs]
    )
    
    print(f"\n**관련 스크립트 부분:**\n{start_at_and_context}")
    print("\n---\n")